### NOTE FOR LUCA

**Remember to set/remove metadata as:**
{
  "nbsphinx": "hidden"
}

to enable/disable solutions view


# Practical 13

In this practical we will continue with object oriented programming and will focus on code testing in Python. Finally, we will introduce regular expressions.

## Slides

The slides of the introduction can be found here: [Intro](docs/Practical13.pdf)

## Testing your code 

Testing the code is quite an important step to make sure that the code is predictable and although some bugs can always slip through, **testing is the process of making the code as predictable as possible**.

Two types of testing exist: the **white-box** testing methodology looks into every detail of the implemented code to inspect its correctness, **black-box** testing does not look at the details of how the code is implemented, but it just focus on the correctness of the output produced by the code.

Testing is quite a complex and articulated topic, here we will just scratch the tip of the iceberg (there are many books on the topic if you are interested).

A **test** is a piece of code written with the sole purpose of checking the correctness of another piece of code. 

Testing requires three successive moments: first of all we need to **set up** (or prepare) the test setting up connections/interfaces to test data, the second step is to **execute** the piece of code we are aiming to test using the interfaces devised at the previous step, and the third is the **verification** of the results to make sure they look as they were expected to.

### Doctest 

A very simple way to specifying tests for the code is by using an embedded module called **doctest**. It will basically search for pieces of code in your python file that look like **interactive python sessions** (that are lines starting with ```>>> ```) and will execute them to check if they run giving the result specified in the next line.

<div class="alert alert-warning">

**Important:** 

Note the space after ">>> ". That is where the test starts. An example:
```
This is a function that returns three values in a list...
>>> fun(x)
[x, y, z]
```
</div>

**Example:**
Let's define some doctest tests for the simple function computing the first N prime numbers.

In [34]:
%reset -f 

def getFirstNprimes(N):
    '''
    This function should output the first N prime numbers.
    >>> getFirstNprimes(1)
    [2]
    >>> getFirstNprimes(2)
    [2, 3]
    >>> getFirstNprimes(10)
    [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
    '''
    if N == 0:
        return []
    res = [2]
    current = 3
    while len(res) < N:
        if len([x for x in res if current % x == 0]) == 0:
            res.append(current)
        current += 1
    #uncomment next line to introduce a bug
    #res.append(1)
    return res        

if __name__ == "__main__":
    import doctest
    doctest.testmod()
    print(getFirstNprimes(20))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]


The line ```if __name__ == "__main__":``` is used to specify if the code is executed as a script (i.e. it is not invoked as an imported module somewhere else in another piece of code).

Executing the code as it is will not give any particular error as the tests we set-up passed correctly. But if we uncomment the ```res.append(1)``` the tests will fail and we produce the following testing output, which reports the three failed tests, the expected values and the obtained values: 

![](img/pract13/doctest.png)

Another way of testing the code is through unit tests, we will see this later on. 

### Raising exceptions and using assertions

One thing we can do is to raise exceptions whenever some pre-conditions are not met in order to insure that these do not lead to erroneous behaviours. This can be done with ```raise Exception("exception text")```. More info on raising exceptions are [here](https://docs.python.org/3/tutorial/errors.html#raising-exceptions). 

**Example**: 
Consider the following ```MyIntPair``` class that works with integers. If we want to make sure it only contains integers we can add a ```raise Exception``` in case it is not an integer. 

In [37]:
class MyIntPair:
    def __init__(self, x,y):
        if not type(x) == int:
            raise Exception("x {} is not integer".format(x))
        if not type(y) == int:
            raise Exception("y {} is not integer".format(y))
        self.x = x
        self.y = y
        
    def __add__(self, other):
        return (self.x + other.x, self.y + other.y)
    
A = MyIntPair(5,10)
B = MyIntPair(3,6)
print(A + B)
C = MyIntPair(1, "two")

(8, 16)


Exception: y two is not integer

Note that if we try to pass it something other than an integer, the execution will stop. To amend this behaviour we need to intercept the exception and deal with them appropriately. This topic is quite articulated, you can find more info on it [here](https://docs.python.org/3/tutorial/errors.html#handling-exceptions).

A simple exception handling can be done by using the ```try - except``` construct that tries to run some statements, being ready to intercept and handle any exception that might occur:

In [43]:
class MyIntPair:
    def __init__(self, x,y):
        if not type(x) == int:
            raise Exception("x: {} is not integer".format(x))
        if not type(y) == int:
            raise Exception("y: {} is not integer".format(y))
        self.x = x
        self.y = y
        
    def __add__(self, other):
        return (self.x + other.x, self.y + other.y)
    
try: 
    A = MyIntPair(5,10)
    B = MyIntPair(3,6)
    #Uncomment to see a different error
    #print(A/0)
    print(A + B)
    C = MyIntPair(1, "two")
    print(A + C)

except Exception as e:
    print("Whoops something went wrong. Ignore the rest.")
    print(str(e))
    

(8, 16)
Whoops something went wrong. Ignore the rest.
y two is not integer


Similarly, we can produce assertions that test if a condition is True, otherwise the execution is stopped with an ```AssertionError```. This error can be caught and handled appropriately. Note that in the code below, only the AssertionError is caught and a division by 0 (e.g. print(10/0))

In [52]:
class MyIntPair:
    def __init__(self, x,y):
        assert type(x) == int, "x: {} is not integer".format(x)
        assert type(y) == int, "y: {} is not integer".format(y)
        self.x = x
        self.y = y
        
        
    def __add__(self, other):
        return (self.x + other.x, self.y + other.y)
    
try: 
    A = MyIntPair(5,10)
    B = MyIntPair(3,6)
    #Uncomment to see a different error (not captured!)
    #print(10/0)
    print(A + B)
    C = MyIntPair(1, "two")
    print(A + C)

except AssertionError as e:
    print("Whoops something went wrong. Ignore the rest.")
    print(str(e))

(8, 16)
Whoops something went wrong. Ignore the rest.
y: two is not integer


## Unit tests

Python comes with a fully-fledgeted testing module which is called ```unittest```. Have a look [here](https://docs.python.org/3.8/library/unittest.html) for detailed information on this module.

The module ```unittest``` must be imported first with ```import unittest``` and then the Test class must be implemented to perform the tests.

In a nutshell, to create some unit tests we need to define a ```Testing``` class (we are free to call it as we like) which is a subclass of the class ```unittest.TestCase```. Within this class, we have then to specify the tests we want to run. Every test is a method and its name **must start** with ```test_``` (e.g. ```test_length```).

Tests can use assertions ```assertEqual(value1, value2)```, ```assertTrue(condition)``` or ```assertFalse(condition)``` that respectively allow to check the equality of two values (i.e. the known result and the output of the method to be tested) and the truth value of a condition (typically computed on the output of the method under test).

We can add the invocation to the ```unittest``` module within the script by adding (```unittest.main()```) for example in the ```main``` part through ```if __name__ == "__main__":```. In this case, we can perform the tests just by calling something like:
```
python3 my_testing_function.py
```

without specifying ```unittest.main()``` in the script, we need to call the unittest with:
```
python3 -m unittest my_testing_function.py
```

The unittest will provide us feedback on how the tests performed. In particular, if all the tests are passed we should get something like:

```
python3 -m unittest file_samples/my_testing_function.py 

.....
----------------------------------------------------------------------
Ran 5 tests in 0.387s

OK
```
otherwise, in case of some errors like:

```
...FF
======================================================================
FAIL: test_one (file_samples.my_testing_function.Testing)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/biancol/Google Drive/work/courses/QCBsciprolab/file_samples/my_testing_function.py", line 33, in test_one
    self.assertEqual(getFirstNprimes(1),[2])
AssertionError: Lists differ: [10] != [2]

First differing element 0:
10
2

- [10]
+ [2]

======================================================================
FAIL: test_ten (file_samples.my_testing_function.Testing)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/biancol/Google Drive/work/courses/QCBsciprolab/file_samples/my_testing_function.py", line 37, in test_ten
    [2, 3, 5, 7, 11, 13, 17, 19, 23, 29])
AssertionError: Lists differ: [2, 3, 5, 7, 11, 13, 17, 10, 23, 29] != [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

First differing element 7:
10
19

- [2, 3, 5, 7, 11, 13, 17, 10, 23, 29]
?                           ^

+ [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
?                           ^


----------------------------------------------------------------------
Ran 5 tests in 0.770s

FAILED (failures=2)

```

**Example:**
Let's define some doctest tests for the simple function computing the first N prime numbers. The file is available here: [my_testing_function.py](file_samples/my_testing_function.py).

In [60]:
%reset -f 
"""
In file my_testing_function.py
"""

import unittest
import random

def getFirstNprimes(N):
    '''
    This function should output the first N prime numbers.
     '''
    if N <= 0:
        return []
    res = [2]
    current = 3
    while len(res) < N:
        if len([x for x in res if current % x == 0]) == 0:
            res.append(current)
        current += 1
    #uncomment next line to introduce a bug
    #res.append(1)
    #or a more subtle error:
    #ind = random.randint(len(res))
    #res[ind] = 10
    return res        

class Testing(unittest.TestCase):
    def test_empty(self):
        self.assertEqual(getFirstNprimes(0),[])
    
    def test_one(self):
        self.assertEqual(getFirstNprimes(1),[2])
    
    def test_ten(self):
        self.assertEqual(getFirstNprimes(10),
                         [2, 3, 5, 7, 11, 13, 17, 19, 23, 29])
    
    def test_len(self):
        for i in range(0,10):
            n = random.randint(1,1000)
            self.assertFalse(len(getFirstNprimes(n)) != n)
    
    
    def test_negative(self):
        self.assertTrue(len(getFirstNprimes(-1)) == 0)

if __name__ == "__main__":
    #uncomment to run the tests in the main (without -m)
    #unittest.main()
    print(getFirstNprimes(20))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]




Running the tests on the previous code without any "intentional bug" will produce an OK message as above, 


## Regular Expressions 

**Example:** Let's align the first 100 bases of the first entry of the file [contigs82.fasta](file_samples/contigs82.fasta) to the Malus Domestica genome.

**NOTE: this can take several minutes.**

In [ ]:
from Bio.PDB import *

pdbl = PDBList()
structures = ["3C2K", "3C2L"]
el = pdbl.download_pdb_files(structures, 
                             file_format = "mmCif", 
                             pdir = "file_samples/")


## Exercises


1. Write a python script that retrieves all the information present in SRA regarding PacBio sequencing performed on E.coli strain K12 (query term is "E.coli K12 wgs PacBio"). Print the number of results and for each id report the title, the accession id, the total number of spots and total number of bases sequenced. 

Sample output: 
```
Entries found: 9
Results for id: 357838
E. coli K12 PacBio RS C2 CCS sequencing
 - acc="SRR799325"
 - total_spots="81741"
 - total_bases="242869038"
Results for id: 357018
E. coli K12 MG1655 PacBio RS C2 Sequencing
 - acc="SRR797943"
 - total_spots="81741"
 - total_bases="207734764"
 
 ...
 ...
```

<div class="tggle" onclick="toggleVisibility('ex2a');">Show/Hide Solution</div>
<div id="ex2a" style="display:none;">

In [ ]:
%reset -f

from Bio import Entrez

Entrez.email = "my_email"
handle = Entrez.esearch(db="sra", term="E.coli K12 wgs PacBio", retmax = 10)
res = Entrez.read(handle)
#uncomment to see all fields:
#for el in res.keys():
#    print(el , " : ", res[el])
#print("Entries found: {}".format(res["Count"]))


for ids in res["IdList"]:    
    print("Results for id:", ids)
    handle = Entrez.esummary(db="sra",  id = ids)
    res = Entrez.read(handle)
    #uncomment to see all info
    #print(res)
    for r in res:
        info = r['ExpXml']
        #print(info)
        runs = r['Runs']
        #some text parsing to do:
        title = info.split("Title>")
        #print(title)
        #print(title[1][:-2])
        print(runs)
        r = runs.split(" ")
        print(r)
        print(" - {}\n - {}\n - {}".format(r[1],
                                           r[2],
                                           r[3]))

</div>

2. Write a python function that reads all the entries of a blast alignment file in .xml format (like [blast_res_apple.xml](file_samples/blast_res_apple.xml) and outputs all the HSPs (see example below) having bitscore > B, alignment length >  A and minimum percentage of identity > I, where B, A and I are input thresholds. Hint: implement a filtering function: *filterHSPs(align, minBitscore = 0, minAlignLen = 0, minPercIdent = 0.1)*.

```
Alignments of MDC020656.85
	MDC020656.85: 1939-2593
	gi|125995253|dbj|AB270792.1|: 201263-201917
	Score:820.917 AlignLen:579 Id/Len:0.8812785388127854
	MDC020656.85: 1446-1935
	gi|125995253|dbj|AB270792.1|: 306490-306017
	Score:582.873 AlignLen:428 Id/Len:0.8629032258064516
    ....
    ....
```

that is reporting the HSP with query start-end position, subject start-end position, score, alignment length and number of identities / alignment length. 

<div class="tggle" onclick="toggleVisibility('ex0');">Show/Hide Solution</div>
<div id="ex0" style="display:none;">

In [ ]:
%reset -f

from Bio.Blast import NCBIXML


def filterHSPs(align, minBitscore = 0, minAlignLen = 0, minPercIdent = 0.1):
    ret = []

    for h in align.hsps:
            b = h.bits
            i = h.identities 
            al = h.align_length
            toOut = ((b > minBitscore) and 
                    (al > minAlignLen) and
                    (i/al > minPercIdent))

            
            
            if(toOut):
                qs = h.query_start
                ss = h.sbjct_start
                qe = h.query_end
                se = h.sbjct_end
                ret.append([qs,qe, ss,se, b, i, al])
             
    return ret        

result_handle = open("file_samples/blast_res_apple.xml")


for res in NCBIXML.parse(result_handle):
    print("Alignments of {}".format(res.query))
    for align in res.alignments:
        filtered = filterHSPs(align, 300, 50, 0.8)
        if(len(filtered) > 0):
            for h in filtered:
                title = align.title.split( )[0]
                print("\t{}: {}-{}".format(res.query,h[0],h[1]))
                print("\t{}: {}-{}".format(title,h[2],h[3]))
                print("\tScore:{} AlignLen:{} Id/Len:{}".format(
                                                                h[4],
                                                                h[5],
                                                                h[5]/h[6]
                                                               ))
            
    

            

result_handle.close()

</div>

3. Write a python function ```retrieve_sequences(search_term, number, outfile)``` that retrieves the first ```number``` of sequences from NCBI's "nucleotide" database having a search  term  ```term``` (hint: use term and retmax parameters of Entrez.esearch) and stores them in a fasta file ```outfile``` (hint: use SeqIO.write). Test your code retrieving the first 5 entries having search term "starch AND Malus Domestica [Organism]"

<div class="tggle" onclick="toggleVisibility('ex1');">Show/Hide Solution</div>
<div id="ex1" style="display:none;">

In [ ]:
%reset -f

from Bio import Entrez
from Bio import SeqIO

def retrieve_sequences(search_term, number, filename):
    Entrez.email = "my_email"
    handle = Entrez.esearch(db="nucleotide", 
                            term=search_term, 
                            retmax=5)
    res = Entrez.read(handle)
    records = []
    for el in res["IdList"]:
        handle = Entrez.efetch(db="nucleotide", 
                               id=el, 
                               rettype = "gb", 
                               retmode="text")
        my_seq = SeqIO.read(handle, format = "genbank")
        records.append(my_seq)
    N = SeqIO.write(records, filename, "fasta")
    print("Search term was: ", search_term)
    print("{} sequences written to {}".format(N,filename))
    
s_term = "starch AND Malus Domestica [Organism]"
retrieve_sequences(s_term, 5, "file_samples/starch_sequences.fasta")


</div>

4. Write a python function that aligns the sequences  in the file created in exercise 3. ([here](file_samples/starch_sequences.fasta) you can find mine) against the NCBI nr database limiting the hits to the Malus Domestica organism (parameter entrez_query='"Malus Domestica" [Organism]' in qblast)and prints to screen the following info for each hsp: 
    1. The title;
    2. Score and e-value;
    3. The number of alignments on the same subject, the number of identities and positives and the alignment length;
    4. The number of mismatches and the list of their positions (hint: you can use the match string and look for " ").
   

    
<div class="tggle" onclick="toggleVisibility('ex2');">Show/Hide Solution</div>
<div id="ex2" style="display:none;">

In [ ]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

fasta_string = open("file_samples/starch_sequences.fasta").read()

res_handle = NCBIWWW.qblast("blastn", "nt", fasta_string, 
                            entrez_query='"Malus Domestica" [Organism]'
                           )

for align in NCBIXML.parse(res_handle):
    
    for a in align.alignments:
            print("Align Title:{}".format(a.title))
            
            for h in a.hsps:
                s = h.score
                e = h.expect
                n = h.num_alignments
                i = h.identities
                p = h.positives
                m = h.match 
                al = h.align_length
                misM = [str(x) for x in range(len(m)) if m[x] == " "]
                print("Score: {} E-val: {}".format(s,e))
                print("N.aligns:{} Ident:{} Pos.:{} Align len:{}".format(
                    n,i,p,al))
                if(len(misM)):
                    print("Num mismatches:",len(misM))
                    print("Mismatch pos:", ",".join(misM))
                else:
                    print("No mismatches")
                print("")
            

res_handle.close()



</div>

5. Write a python function ```getPublicationInfo(title_term,other_term)``` that retrieves the first 20 pubmed publications having the ```title_term``` in the title and ```other_term``` somewhere else in the text (hint use: "Title" and "[Other Term]" as esearch parameter term). For each publication print: 
    1. the title
    2. authors 
    3. journal 
    4. year of publication (hint: get and split properly the "PubDate" entry)
    5. a link to the pubmed entry (hint: it is the string "https://www.ncbi.nlm.nih.gov/pubmed/" followed by the pubmed id ("eid" entry of the dictionary "ArticleIds"). es: https://www.ncbi.nlm.nih.gov/pubmed/26919684

Hint: to see how to combine search terms test them here: [https://www.ncbi.nlm.nih.gov/pubmed/advanced](https://www.ncbi.nlm.nih.gov/pubmed/advanced).

Test your code calling ```getPublicationInfo("apple","drought")```

<div class="tggle" onclick="toggleVisibility('ex3');">Show/Hide Solution</div>
<div id="ex3" style="display:none;">

In [ ]:

from Bio import Entrez

def getPublicationInfo(title_term,other_term): 
    Entrez.email = "my_email"
    s_term = title_term + " [Title] AND " + other_term + " [Other Term]"
    handle = Entrez.esearch(db="pubmed", term=s_term)
    res = Entrez.read(handle)
#uncomment to see all info
#    for el in res.keys():
#        print(el , " : ", res[el])
#
#    print("")
    for ids in res["IdList"]:    
        handle = Entrez.esummary(db="pubmed",  id = ids)
        res = Entrez.read(handle)
        #uncomment to see all info
        #print(res)
        for r in res:
            print(r["Title"])
            print(",".join(r["AuthorList"]))
            print(r["Source"])
            print(r["PubDate"].split()[0])
            print("https://www.ncbi.nlm.nih.gov/pubmed/" + r["ArticleIds"]["eid"])
            print("")
            
getPublicationInfo("apple","drought")

</div>

6. Write some python code to retrieve the structure of two forms of the aspartate transcarbamoylase (PDB ids: 4FYW and 1D09). If you are interested, read more about the Aspartate Transcarbamoylase [here](http://pdb101.rcsb.org/motm/215). Write a function that gets the .cif file name and prints:

    1. the number of chains, residues and atoms present in the file;
    2. a histogram of the residues (plotting it with matplotlib) that are not water (encoded as "HOH");
    3. a link to an online tool to visualize the 3D structure. The link will be "http://www.rcsb.org/pdb/ngl/ngl.do?pdbid=" followed by the PDB id of the protein (e.g. 1d09).

<div class="tggle" onclick="toggleVisibility('ex4');">Show/Hide Solution</div>
<div id="ex4" style="display:none;">

In [ ]:
from Bio.PDB import *
import matplotlib.pyplot as plt

def printCifInfo(filename):
    
    parser = MMCIFParser(QUIET=True) #To disable warnings
    id = filename.split("/")[1].split(".")[0]

    structure = parser.get_structure(id, filename)
    chains = structure.get_chains()
    residues = structure.get_residues()
    
    atoms = structure.get_atoms()
    res_histo = {}
    resCnt = 0 #need this because while reading the residues 
               #I am pulling stuff out of the iterator
    for res in residues:
        rname = res.get_resname()
        if(rname != "HOH"):
            if( rname not in res_histo):
                res_histo[rname] = 1
            else:
                res_histo[rname] += 1
        resCnt += 1    
    plt.figure(figsize=(15,5))
    plt.bar(res_histo.keys(), res_histo.values())
    plt.show()
    print("Number of chains: {}".format(len(list(chains))))
    print("Number of residues: {}".format(resCnt))
    print("Number of atoms: {}".format(len(list(atoms))))
    print("http://www.rcsb.org/pdb/ngl/ngl.do?pdbid=" + id) 

pdbl = PDBList()
structures = ["1D09", "4FYW"]
el = pdbl.download_pdb_files(structures, file_format = "mmCif", pdir = "file_samples/")

printCifInfo("file_samples/1d09.cif")
printCifInfo("file_samples/4fyw.cif")

In [ ]:
</div>